## **MÓDULO 5 - APLICACION WEB DE CIENCIA DE DATOS**

---

**DS&AI: Reto del MODULO 5**

NOMBRE: EDGAR ARMANDO OLIVO RODRIGUEZ

> Reto de aplicación sobre la hipótesis de deserción laboral provista por el Hackathon HackerEarth 2020 complementando el análisis de los datos con la representación en un dashboard usando **Streamlit**.

1. **Libreta en Google Colab para el reto.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


2.	**Instalación de streamlit, ngrok y crea el túnel de comunicación para visualizar la aplicación.**

In [2]:
# Instalar de la librería streamlit
!pip install streamlit
# Cargar máquina virtual que tenga instalado Ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# Cargar máquina virtual en el archivo de Google Colab
!unzip ngrok-stable-linux-amd64.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.1 MB 7.5 MB/s 
     |████████████████████████████████| 78 kB 6.2 MB/s 
     |████████████████████████████████| 4.7 MB 40.6 MB/s 
     |████████████████████████████████| 181 kB 39.1 MB/s 
     |████████████████████████████████| 235 kB 66.5 MB/s 
     |████████████████████████████████| 164 kB 61.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 51 kB 7.4 MB/s 
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=cff7ea0c382bbcda2dc38b5a419c543f65488fa5f84c22b836d4df7915dfc9dd
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators
--2022-09-12 20:23:04--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.8

In [3]:
# Registar el Token  para visualizar la aplicación
get_ipython().system_raw('./ngrok authtoken 2E3EBl6bT6M69JsFTQ5DhMTKDbg_2tJ9PrTTr7yTnpDTaqny9')

In [4]:
# Establecer puerto 8501 a Ngrok para visualizar la aplicación
get_ipython().system_raw('./ngrok http 8501 &')

In [5]:
# Crear el tunel para generar el enlace de vusualización de la aplicación
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://b1ca-34-125-209-55.ngrok.io


3.	**Creación de la Aplicación**

>> Descarga el archivo: Employees.csv y guarda, en un dataframe (`employees`), para evitar tráfico innecesario construir una función principal donde recuperen 'n' datos (`500` por `default`) para la etapa de pruebas y desarrollo, NOTA: usar el atributo cache.

>> Observa la estructura y contenido del dataframe con los atributos.

>> Crear el programa `employees.py` que contendrá la aplicación (usar `%%writefile`).


In [6]:
%%writefile employees.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import streamlit as st
import codecs

DATA_URL = '/content/Employees.csv'
E_ID = 'Employee_ID'

# Definición de funciones
@st.cache
def load_data(nrows):
    doc = codecs.open(DATA_URL,'rU','latin1')
    data = pd.read_csv(doc, nrows=nrows)
    lowercase = lambda x: str(x).lower()
    return data

def filter_by_id(id):
    dfid = data[data[E_ID] == id]
    return dfid

def filter_by_location(location,col):
    filterbyloc = data[data[col].str.contains(location, case=False)]
    return filterbyloc

def filter_by_type(tipo,col):
    filterbytype = data[data[col] == tipo]
    return filterbytype

def gen_graph(gtype, attribute1, attribute2, bins, color, gtitle, xlabel, ylabel):
    fig, ax = plt.subplots()
    if gtype == 'hist':
        ax.hist(attribute1, bins=bins, color=color)
    elif gtype == 'barh':
        ax.barh(attribute1, attribute2, color=color)
    ax.set_title(gtitle)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    return st.pyplot(fig)

# Creación del título de la aplicación, encabezados y texto de descripción del proyecto. 
st.title("Employees Dashboard")
st.header("DS&AI Reto M5")

st.write("""
Fenómeno de deserción laboral en empresas y organizaciones

Fuente de la hipótesis: Datos del Hackathon HackerEarth 2020.
""")

# Creación de la barra lateral
sidebar = st.sidebar
sidebar.title("Filtros:")

# Recuperar registros
count_rows = 0
DATA_QUOTA = sidebar.slider('Selecciar cantidad de registros del análisis', 0, 7000, 500, step=500)
data_load_state = sidebar.text('Loading Employees Data...')
data = load_data(DATA_QUOTA)
data_load_state.text("Done! (using s.cache)")
count_rows = data.shape[0]

if count_rows > 0:
    if sidebar.checkbox("Mostrar todas los empleados"): # Creación del CHECKBOX mostrar dataframe
       st.write("Total de Empleados:", count_rows)
       st.write(data)
    
    st.markdown("___")

    # TEXT_INPUT+BUTTON del buscador por Empleado, por Ciudad de Origen y por Unidades
    employee_id = sidebar.text_input('ID Empleado: ')
    id_btn = sidebar.button('BUSCAR EMPLEADO')
    if (id_btn):
        employee = filter_by_id(employee_id)
        if employee.shape[0] != 0:
            st.write(f"Registro del empleado con ID {employee_id}:")
        else:
            st.write(f"El ID {employee_id} no se encuantra en los datos!")
        st.write(employee)

    hometown = sidebar.text_input('Ciudad de Origen: ')
    hometown_btn = sidebar.button('FILTRAR X CIUDAD')
    if (hometown_btn):
        dfhometown = filter_by_location(hometown,'Hometown')
        st.write(f"{hometown} tiene una coincidencia de {dfhometown.shape[0]} de los {count_rows} registros de la muestra")
        st.write(dfhometown)

    unit = sidebar.text_input('Unidad: ')
    unit_btn = sidebar.button('FILTRAR X UNIDAD')
    if (unit_btn):
        dfunit = filter_by_location(unit, 'Unit')
        st.write(f"{unit} tiene una coincidencia de {dfunit.shape[0]} de los {count_rows} registros de la muestra")
        st.write(dfunit)

    st.markdown("___")

    # SELECTBOX del filtro de Empleados por Nivel Educativo / por Ciudad / y por Unidad Funcional
    education_level = sidebar.selectbox("Seleccionar Nivel Educativo", data['Education_Level'].unique(), index=2)
    if (education_level):
        dfeducatiolevel = filter_by_type(education_level,'Education_Level')
        st.write(f"{dfeducatiolevel.shape[0]} empleados en el nivel educativo [{education_level}]")
        st.write(dfeducatiolevel)

    city = sidebar.selectbox("Seleccionar Ciudad", np.append(['----'], data['Hometown'].unique()))
    if (city):
        if city != '----':
            dfcity = filter_by_type(city, 'Hometown')
            st.write(f"{dfcity.shape[0]} empleados en {city}")
            st.write(dfcity)

    funit = sidebar.selectbox("Seleccionar Unidad Funcional", np.append(['----'], data['Unit'].unique()))
    if (funit):
        if funit != '----':
            dffu = filter_by_type(funit, 'Unit')
            st.write(f"{dffu.shape[0]} empleados en la unidad funcional {funit}")
            st.write(dffu)

    st.markdown("___")

    # PRESENTACION DE GRAFICAS
    st.subheader("Visualización de Datos")

    gen_graph('hist', data.Age, '', [20,25,30,35,40,45,50,55,60,65,70], 'lightskyblue', 'Histograma de empleados agrupados por edad', 'Rangos de Edades', 'Cantidad de Empleados') # HISTOGRAMA por Edad, usando funciones

    st.markdown("___")

    empbyunit = data.groupby(by=["Unit"]).agg({"Unit":np.size})
    empbyunit = empbyunit.rename(columns = {"Unit" : "Total"})
    eubd = empbyunit.reset_index()

    gen_graph('barh', eubd.Unit, eubd.Total, 0, 'turquoise', 'Frecuencia de empleados por unidades funcionales', 'Cantidad de Empleados', '') # GRAFICA DE FRECUENCIAS de Unidades Funcionales, usando Funciones

    st.markdown("___")

    fig, axs = plt.subplots(3,1,figsize=(5,15))
    plt.subplots_adjust(hspace=0.3)
            # Ciudades con mayor índice de deserción
    axs[0].bar(data.Hometown, data.Attrition_rate, color='plum')
    axs[0].set_title('Índice de deserción por ciudad')
    axs[0].set_ylabel("Tasa de Deserción")
            # Edad e índice de deserción
    axs[1].scatter(data.Age, data.Attrition_rate, color='purple')
    axs[1].set_title('Índice de deserción por edad')
    axs[1].set_xlabel("Edad")
    axs[1].set_ylabel("Índice de Deserción")
            # Ciudades con mayor índice de deserción
    axs[2].scatter(data.Time_of_service, data.Attrition_rate, color='purple')
    axs[2].set_title('Índice de deserción por tiempo de servicio')
    axs[2].set_xlabel("Tiempo de Servicio")
    axs[2].set_ylabel("Índice de Deserción")

    st.pyplot(fig)

    st.markdown("___")

    st.write("""
    * Asociación NULA entre las variables numéricas.
      No existe correlación entre el Índice de Deserción y la Edad,
      ni entre el Índice de Deserción y el Tiempo de Servicio.
    """)
else:
    st.warning("0 registros; Indique la cantidad de datos para el análisis!")

Writing employees.py


4.	**Ejecución de la Aplicación**

In [ ]:
# Ejecutar la apliación con el comando streamlit
!streamlit run /content/employees.py

2022-09-12 20:24:01.566 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.125.209.55:8501

